# Packages

In [8]:
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
from tifffile import imread, imsave, imwrite
import math
import os, shutil
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, LeakyReLU, Conv3D, ZeroPadding3D, MaxPooling3D, UpSampling3D, concatenate, Conv3DTranspose, BatchNormalization, Dropout, Lambda
from tensorflow.keras.optimizers import Adam
from keras.layers import Activation, MaxPool2D, Concatenate
from skimage import io
import numpy as np
from matplotlib import pyplot as plt
from keras import backend as K
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from tifffile import imread
import math
import xlwt
from sys import stdout
from xlwt import Workbook
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from torch.nn import ReflectionPad3d
import torch
from matplotlib.pyplot import imshow

# Models

In [9]:
def Generator(img_shape):
    '''
    Generator model
    '''
    def encoder_step(layer, Nf, ks, norm=True):
        x = Conv3D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(layer)
        if norm:
            x = InstanceNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(0.2)(x)

        return x

    def bottlenek(layer, Nf, ks):
        x = Conv3D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(layer)
        x = InstanceNormalization()(x)
        x = LeakyReLU()(x)
        for i in range(4):
            y = Conv3D(Nf, kernel_size=ks, strides=1, kernel_initializer='he_normal', padding='same')(x)
            x = InstanceNormalization()(y)
            x = LeakyReLU()(x)
            x = Concatenate()([x, y])

        return x

    def decoder_step(layer, layer_to_concatenate, Nf, ks):
        x = Conv3DTranspose(Nf, kernel_size=ks, strides=2, padding='same', kernel_initializer='he_normal')(layer)
        x = InstanceNormalization()(x)
        x = LeakyReLU()(x)
        x = Concatenate()([x, layer_to_concatenate])
        x = Dropout(0.2)(x)
        return x

    layers_to_concatenate = []
    inputs = Input((64,64,64,2), name='input_image')
    Nfilter_start = 64
    depth = 4
    ks = 4
    x = inputs

    # encoder
    for d in range(depth-1):
        if d==0:
            x = encoder_step(x, Nfilter_start*np.power(2,d), ks, False)
        else:
            x = encoder_step(x, Nfilter_start*np.power(2,d), ks)
        layers_to_concatenate.append(x)

    # bottlenek
    x = bottlenek(x, Nfilter_start*np.power(2,depth-1), ks)

    # decoder
    for d in range(depth-2, -1, -1): 
        x = decoder_step(x, layers_to_concatenate.pop(), Nfilter_start*np.power(2,d), ks)

    # classifier
    last = Conv3DTranspose(2, kernel_size=ks, strides=2, padding='same', kernel_initializer='he_normal', activation='sigmoid', name='output_generator')(x)
   
    return Model(inputs=inputs, outputs=last, name='Generator')

def Discriminator(img_shape, mask_shape):
    '''
    Discriminator model
    '''

    inputs = Input(img_shape, name='input_image')
    targets = Input(mask_shape, name='target_image')
    Nfilter_start = 64
    depth = 3
    ks = 4

    def encoder_step(layer, Nf, norm=True):
        x = Conv3D(Nf, kernel_size=ks, strides=2, kernel_initializer='he_normal', padding='same')(layer)
        if norm:
            x = InstanceNormalization()(x)
        x = LeakyReLU()(x)
        x = Dropout(0.2)(x)
        
        return x

    x = Concatenate()([inputs, targets])

    for d in range(depth):
        if d==0:
            x = encoder_step(x, Nfilter_start*np.power(2,d), False)
        else:
            x = encoder_step(x, Nfilter_start*np.power(2,d))
            
    x = ZeroPadding3D()(x)
    x = Conv3D(Nfilter_start*(2**depth), ks, strides=1, padding='valid', kernel_initializer='he_normal')(x) 
    x = InstanceNormalization()(x)
    x = LeakyReLU()(x)
      
    x = ZeroPadding3D()(x)
    last = Conv3D(1, ks, strides=1, padding='valid', kernel_initializer='he_normal', name='output_discriminator')(x) 

    return Model(inputs=[inputs, targets], outputs=last, name='Discriminator')


# Losses

In [10]:

def diceLoss(y_true, y_pred):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, y_true.dtype)

    num = tf.math.reduce_sum(tf.math.multiply(y_true, y_pred))
    den = tf.math.reduce_sum(tf.math.add(y_true, y_pred))+1e-5

    return 1-2*num/den

def weighted_diceLoss(y_true,y_pred, class_weight=[6.63,0.5]):
    
    weighted_loss = class_weight[0]*diceLoss(y_true[:,:,:,:,0], y_pred[:,:,:,:,0])+class_weight[1]*diceLoss(y_true[:,:,:,:,1], y_pred[:,:,:,:,1])
    loss = weighted_loss/np.sum(class_weight)
    
    return loss
    

def discriminator_loss(disc_real_output, disc_fake_output):
    real_loss = tf.math.reduce_mean(tf.math.pow(tf.ones_like(disc_real_output) - disc_real_output, 2))
    fake_loss = tf.math.reduce_mean(tf.math.pow(tf.zeros_like(disc_fake_output) - disc_fake_output, 2))

    disc_loss = 0.5*(real_loss + fake_loss)

    return disc_loss


def generator_loss(target, gen_output, disc_fake_output, alpha):
    
    # generalized dice loss
    dice_loss = diceLoss(target, gen_output)
    
    # disc loss
    disc_loss = tf.math.reduce_mean(tf.math.pow(tf.ones_like(disc_fake_output) - disc_fake_output, 2))
       
    # total loss
    gen_loss = alpha*dice_loss + disc_loss

    return gen_loss, dice_loss, disc_loss

# Training

In [11]:
# Models 
img_shape = (64,64,64,2)
mask_shape = (64,64,64,2)

G = Generator(img_shape)
D = Discriminator(img_shape, mask_shape)

# Optimizers
generator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(2e-4, beta_1=0.5)


def train_step(image, target, alpha):

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:

            gen_output = G(image, training=True)

            disc_real_output = D([image, target], training=True)
            disc_fake_output = D([image, gen_output], training=True)
            disc_loss = discriminator_loss(disc_real_output, disc_fake_output)
            
            gen_loss, dice_loss, disc_loss_gen = generator_loss(target, gen_output, disc_fake_output, alpha)

    generator_gradients = gen_tape.gradient(gen_loss, G.trainable_variables)
    discriminator_gradients = disc_tape.gradient(disc_loss, D.trainable_variables)

    generator_optimizer.apply_gradients(zip(generator_gradients, G.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(discriminator_gradients, D.trainable_variables))
        
    return gen_loss, dice_loss, disc_loss_gen


def test_step(image, target, alpha):
    gen_output = G(image, training=False)

    disc_real_output = D([image, target], training=False)
    disc_fake_output = D([image, gen_output], training=False)
    disc_loss = discriminator_loss(disc_real_output, disc_fake_output)

    gen_loss, dice_loss, disc_loss_gen = generator_loss(target, gen_output, disc_fake_output, alpha)
        
    return gen_loss, dice_loss, disc_loss_gen


def thresholding(patch_prediction):
    patch_ind_nuclei = np.argwhere(patch_prediction[:,:,:,0] > 0.5)
    patch_ind_golgi = np.argwhere(patch_prediction[:,:,:,1] > 0.5)

    patch_prediction_thr = np.zeros((patch_prediction.shape[0],patch_prediction.shape[1],patch_prediction.shape[2],3))

    for i in range(patch_ind_nuclei.shape[0]):
      patch_prediction_thr[patch_ind_nuclei[i,0],patch_ind_nuclei[i,1],patch_ind_nuclei[i,2],0]=1 

    for j in range(patch_ind_golgi.shape[0]):
      patch_prediction_thr[patch_ind_golgi[j,0],patch_ind_golgi[j,1],patch_ind_golgi[j,2],1]=1

    return patch_prediction_thr 


def fit(train_gen, valid_gen, alpha, epochs):

  path = './Dataset/Results_Pix2Pix'
  if os.path.exists(path)==False:
    os.mkdir(path)

  
  Nt = len(train_gen)
  history = {'train': [], 'valid': []}
  prev_loss = np.inf
  n = 0
    
  epoch_v2v_loss = tf.keras.metrics.Mean()
  epoch_dice_loss = tf.keras.metrics.Mean()
  epoch_disc_loss = tf.keras.metrics.Mean()
  epoch_v2v_loss_val = tf.keras.metrics.Mean()
  epoch_dice_loss_val = tf.keras.metrics.Mean()
  epoch_disc_loss_val = tf.keras.metrics.Mean()

  for e in range(epochs):
    print('Epoch {}/{}'.format(e+1,epochs))
    b = 0
    
    for Xb, yb in train_gen:
        b += 1

        losses = train_step(Xb, yb[:,:,:,:,:2], alpha)
        epoch_v2v_loss.update_state(losses[0])
        epoch_dice_loss.update_state(losses[1])
        epoch_disc_loss.update_state(losses[2])
        
        stdout.write('\rBatch: {}/{} - loss: {:.4f} - dice_loss: {:.4f} - disc_loss: {:.4f}'
                      .format(b, Nt, epoch_v2v_loss.result(), epoch_dice_loss.result(), epoch_disc_loss.result()))
        stdout.flush()
    history['train'].append([epoch_v2v_loss.result(), epoch_dice_loss.result(), epoch_disc_loss.result()])
    
    for Xb, yb in valid_gen:
        losses_val = test_step(Xb, yb[:,:,:,:,:2], alpha)
        epoch_v2v_loss_val.update_state(losses_val[0])
        epoch_dice_loss_val.update_state(losses_val[1])
        epoch_disc_loss_val.update_state(losses_val[2])
            
    stdout.write('\n               loss_val: {:.4f} - dice_loss_val: {:.4f} - disc_loss_val: {:.4f}'
                  .format(epoch_v2v_loss_val.result(), epoch_dice_loss_val.result(), epoch_disc_loss_val.result()))
    stdout.flush()
    history['valid'].append([epoch_v2v_loss_val.result(), epoch_dice_loss_val.result(), epoch_disc_loss_val.result()])
    
    # save pred image at epoch e
    #y_pred = G.predict(Xb)
    #pred_patch = y_pred[0,:,:,:,:]
    #pred_patch_thr = thresholding(pred_patch)

    #predicted_mask = pred_patch_thr*255.0
    #predicted_mask = predicted_mask.astype('uint8')

    #imwrite(path + '/pred_mask_' + str(e+1) , predicted_mask, photometric='rgb')

    # save models 
    path_models = './Models_Pix2Pix'
    if os.path.exists(path_models)==False:
      os.mkdir(path_models)

    print(' ')
    if epoch_v2v_loss_val.result() < prev_loss:    
        G.save_weights(path_models + '/Generator.h5') 
        D.save_weights(path_models + '/Discriminator.h5')
        print("Validation loss decresaed from {:.4f} to {:.4f}. Models' weights are now saved.".format(prev_loss, epoch_v2v_loss_val.result()))
        prev_loss = epoch_v2v_loss_val.result()
        n = 0
    else:
        print("Validation loss did not decrese from {:.4f}.".format(prev_loss))
        # Early Stopping
        n = n+1
        if n > 20:
            return history
    print(' ')

    # resets losses states
    epoch_v2v_loss.reset_states()
    epoch_dice_loss.reset_states()
    epoch_disc_loss.reset_states()
    epoch_v2v_loss_val.reset_states()
    epoch_dice_loss_val.reset_states()
    epoch_disc_loss_val.reset_states()
    
    del Xb, yb
        
  return history


# Main

In [12]:
import sys  
sys.path.insert(0, './')

In [13]:
import data_generator
from data_generator import DataGenerator

In [14]:
alpha=5
n_epochs = 200
batch_size = 4

traingen = DataGenerator('Train', batch_size=batch_size, img_size=(64,64,64,2), mask_size=(64,64,64,2))

valgen = DataGenerator('Validation', batch_size=batch_size, img_size=(64,64,64,2), mask_size=(64,64,64,2))

# train the vox2vox model
h = fit(traingen, valgen, alpha, n_epochs)

Epoch 1/200


2022-10-23 17:15:09.150553: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8101
2022-10-23 17:15:09.303204: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-23 17:15:09.303568: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-23 17:15:09.303588: W tensorflow/stream_executor/gpu/asm_compiler.cc:77] Couldn't get ptxas version string: Internal: Couldn't invoke ptxas --version
2022-10-23 17:15:09.303992: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory
2022-10-23 17:15:09.304045: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: Failed to launch ptxas
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.


Batch: 73/73 - loss: 8.2937 - dice_loss: 0.9367 - disc_loss: 3.610422
               loss_val: 4.5996 - dice_loss_val: 0.8465 - disc_loss_val: 0.3669 
Validation loss decresaed from inf to 4.5996. Models' weights are now saved.
 
Epoch 2/200
Batch: 73/73 - loss: 3.6775 - dice_loss: 0.5891 - disc_loss: 0.7320
               loss_val: 3.1427 - dice_loss_val: 0.3621 - disc_loss_val: 1.3323 
Validation loss decresaed from 4.5996 to 3.1427. Models' weights are now saved.
 
Epoch 3/200
Batch: 73/73 - loss: 2.1054 - dice_loss: 0.3264 - disc_loss: 0.4734
               loss_val: 1.9387 - dice_loss_val: 0.2834 - disc_loss_val: 0.5216 
Validation loss decresaed from 3.1427 to 1.9387. Models' weights are now saved.
 
Epoch 4/200
Batch: 73/73 - loss: 1.7511 - dice_loss: 0.2687 - disc_loss: 0.4074
               loss_val: 1.3810 - dice_loss_val: 0.2504 - disc_loss_val: 0.1290 
Validation loss decresaed from 1.9387 to 1.3810. Models' weights are now saved.
 
Epoch 5/200
Batch: 73/73 - loss: 1.5816 -

In [ ]:
def model_plots(history, plot_dir):
  #plot the training and validation IoU and loss at each epoch
    loss = np.array(history['train'][:])[:,1]
    val_loss = np.array(history['valid'][:])[:,1]
    epochs = range(1, len(loss) + 1)
    plt.plot(epochs, loss, 'y', label='Training loss')
    plt.plot(epochs, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig(plot_dir + '/loss_plot.png')
    plt.clf()

  #acc = history.history['dice_coefficient']
  #val_acc = history.history['val_dice_coefficient']

  #plt.plot(epochs, acc, 'y', label='Training Dice')
  #plt.plot(epochs, val_acc, 'r', label='Validation Dice')
  #plt.title('Training and validation Dice')
  #plt.xlabel('Epochs')
  #plt.ylabel('Dice')
  #plt.legend()
  #plt.savefig(plot_dir + '/dice_plot.png')

In [18]:
plot_dir = './Dataset/Model_Plots_Pix2Pix'
if os.path.exists(plot_dir)==False:
    os.mkdir(plot_dir)

model_plots(h, plot_dir)

<Figure size 432x288 with 0 Axes>

# Test

In [15]:
def padding(image,size):

    img_reshape = np.moveaxis(image, -1, 0)
    
    m = ReflectionPad3d((0,size[2]-image.shape[2],0,size[1]-image.shape[1],0,size[0]-image.shape[0]))
    input = torch.tensor(img_reshape, dtype=torch.float)
    output = m(input)
    pad_img = output.numpy()
    pad_img = np.moveaxis(pad_img, 0, -1)
    
    return pad_img

In [16]:
def thresholding(patch_prediction):
  patch_ind_nuclei = np.argwhere(patch_prediction[:,:,:,0] > 0.5)
  patch_ind_golgi = np.argwhere(patch_prediction[:,:,:,1] > 0.5)

  patch_prediction_thr = np.zeros((patch_prediction.shape[0],patch_prediction.shape[1],patch_prediction.shape[2],3))

  for i in range(patch_ind_nuclei.shape[0]):
    patch_prediction_thr[patch_ind_nuclei[i,0],patch_ind_nuclei[i,1],patch_ind_nuclei[i,2],0]=1 

  for j in range(patch_ind_golgi.shape[0]):
    patch_prediction_thr[patch_ind_golgi[j,0],patch_ind_golgi[j,1],patch_ind_golgi[j,2],1]=1
    
  return patch_prediction_thr

In [17]:
def pred_mask(image, mask_shape):
  patch_size = 64
  step = 48

  pred_mask = np.zeros(((image.shape[0] // step)*step + patch_size,(image.shape[1] // step)*step + patch_size,64,3))
  _image = padding(image, ((image.shape[0] // step)*step + patch_size,(image.shape[1] // step)*step + patch_size,64,3))

  i = 0
  while i + patch_size <= _image.shape[0]:
    j = 0
    while j + patch_size <= _image.shape[1]:

        tst_patch = _image[i:i+patch_size, j:j+patch_size, :, :]
        tst_patch = np.array([tst_patch / 255.])
        preds_tst = G(tst_patch, training=False)
        pred_patch = preds_tst[0,:,:,:,:]
        pred_patch_thr = thresholding(pred_patch)

        pred_mask[i:i+patch_size, j:j+patch_size,:,0] = np.array(np.logical_or(pred_mask[i:i+patch_size, j:j+patch_size, :,0], pred_patch_thr[:,:,:,0], dtype = 'float64'))
        pred_mask[i:i+patch_size, j:j+patch_size,:,1] = np.array(np.logical_or(pred_mask[i:i+patch_size, j:j+patch_size, :,1], pred_patch_thr[:,:,:,1], dtype = 'float64'))
        pred_mask[i:i+patch_size, j:j+patch_size,:,2] = 0

        j += step
        
    i += step
    
  _pred_mask = np.zeros(mask_shape)
  _pred_mask = pred_mask[0:mask_shape[0],0:mask_shape[1],0:mask_shape[2],:]

  return _pred_mask

In [18]:
def dice_coefficient(y_true, y_pred):
    smoothing_factor = 1
    flat_y_true = K.flatten(y_true)
    flat_y_pred = K.flatten(y_pred)
    return (2. * K.sum(flat_y_true * flat_y_pred) + smoothing_factor) / (K.sum(flat_y_true) + K.sum(flat_y_pred) + smoothing_factor)

def precision(y_true, y_pred):
    flat_y_true = K.flatten(y_true)
    flat_y_pred = K.flatten(y_pred)
    return(K.sum(flat_y_true * flat_y_pred) / K.sum(flat_y_pred) )

def recall(y_true, y_pred):
    flat_y_true = K.flatten(y_true)
    flat_y_pred = K.flatten(y_pred)
    return(K.sum(flat_y_true * flat_y_pred) / K.sum(flat_y_true) )

In [19]:
def metrics(mask, _pred_mask):
  _mask = mask/255.

  dice_coef_nuclei = dice_coefficient(_mask[:,:,:,1],_pred_mask[:,:,:,1])
  dice_coef_golgi = dice_coefficient(_mask[:,:,:,0],_pred_mask[:,:,:,0])
  prec_nuclei = precision(_mask[:,:,:,1],_pred_mask[:,:,:,1])
  prec_golgi = precision(_mask[:,:,:,0],_pred_mask[:,:,:,0])
  recall_nuclei = recall(_mask[:,:,:,1],_pred_mask[:,:,:,1])
  recall_golgi = recall(_mask[:,:,:,0],_pred_mask[:,:,:,0])

  return [round(dice_coef_nuclei.numpy(),4), round(dice_coef_golgi.numpy(),4), 
          round(prec_nuclei.numpy(),4), round(prec_golgi.numpy(),4), round(recall_nuclei.numpy(),4),
          round(recall_golgi.numpy(),4)]


In [20]:
def write_to_excel(wb,sheet_name,metrics, result_dir):

  # add_sheet is used to create sheet.
  sheet1 = wb.add_sheet(sheet_name)

  sheet1.write(0, 0, 'Dice Coeffient Nuclei')
  sheet1.write(0, 1, 'Dice Coeffient Golgi')
  sheet1.write(0, 2, 'Precision Nuclei')
  sheet1.write(0, 3, 'Precision Golgi')
  sheet1.write(0, 4, 'Recall Nuclei')
  sheet1.write(0, 5, 'Recall Golgi')
  sheet1.write(1, 0, metrics[0])
  sheet1.write(1, 1, metrics[1])
  sheet1.write(1, 2, metrics[2])
  sheet1.write(1, 3, metrics[3])
  sheet1.write(1, 4, metrics[4])
  sheet1.write(1, 5, metrics[5])

  wb.save(result_dir + '/results_metrics.xlsx')

In [21]:
def test_model(base_dir):

  fnames = os.listdir(base_dir + 'Images/')

  # Workbook is created
  wb = Workbook()

  result_dir = './Dataset/Results_Pix2Pix'
  if os.path.exists(result_dir)==False:
    os.mkdir(result_dir)

  for i in range(len(fnames)):

    image = imread(base_dir + 'Images/' + fnames[i])[:,:,:,:2]
    mask = imread(base_dir + 'Masks/' + fnames[i])

    predicted_mask = pred_mask(image, image.shape)

    _predicted_mask = predicted_mask*255.0
    _predicted_mask = _predicted_mask.astype('uint8')

    imwrite(result_dir + '/pred_mask_' + fnames[i] , _predicted_mask, photometric='rgb')

    _metrics = metrics(mask, predicted_mask)

    write_to_excel(wb,'Sheet_' + fnames[i].split('.')[0], _metrics, result_dir)

In [22]:
import time
start_time = time.time()
#Load the pretrained model for testing and predictions. 

img_shape=(64,64,64,2)
G = Generator(img_shape)
G.load_weights('./Models_Pix2Pix/Generator.h5')

base_dir = './Dataset/Patches_Pre_64/Test/'
test_model(base_dir)

print("--- %s seconds ---" % (time.time() - start_time))

--- 35.63827729225159 seconds ---


# Gerar mascaras

In [48]:
def generate_synthetic_masks(train_dir_img, train_dir_mask, n_patches):

  size_img = (128,128,64,3)
  size_mask = (128,128,64,2)

  for p in range(n_patches):
      img = np.zeros(size_img)
      img[:,:,:,0] = np.random.uniform(low=10, high=20, size=(128,128,64))
      img[:,:,:,1] = np.random.uniform(low=5, high=15, size=(128,128,64))
      img = img/255.
      imwrite(train_dir_img + '/img_patch_' + str(5) + str(p) + '.tif', img)

  for k in range(n_patches):
      patch = np.zeros(size_mask)
      patch = patch/255.
      imwrite(train_dir_mask + '/img_patch_' + str(5) + str(k) + '.tif', patch)


In [49]:
generate_synthetic_masks('./Dataset/Patches_128/Train/Images', './Dataset/Patches_128/Train/Masks', 12)